In [2]:
import blpapi
from optparse import OptionParser
#from optparse import OptionParser
import time

In [3]:
TOKEN_SUCCESS = blpapi.Name("TokenGenerationSuccess")
TOKEN_FAILURE = blpapi.Name("TokenGenerationFailure")
AUTHORIZATION_SUCCESS = blpapi.Name("AuthorizationSuccess")
AUTHORIZATION_FAILURE = blpapi.Name("AuthorizationFailure")
RESPONSE_ERROR = blpapi.Name("responseError")

In [4]:
g_identity = None
g_identityCId = None


In [5]:
class SessionEventHandler(object):
    def getTimeStamp(self):
        return time.strftime("%Y/%m/%d %X")

    def processMiscEvents(self, event):
        timeStamp = self.getTimeStamp()
        for msg in event:
            print ("%s: %s" % (timeStamp, msg.messageType()))
            print (msg)

    def processEvent(self, event, session):
        try:
            return self.processMiscEvents(event)
        except blpapi.Exception as e:
            print ("Library Exception !!! %s" % e.description())
        return False


In [6]:
def parseCmdLine():
    parser = OptionParser(description="Retrieve reference data.")
    parser.add_option("-a",
                      "--ip",
                      dest="host",
                      help="server name or IP (default: %default)",
                      metavar="ipAddress",
                      default="localhost")
    parser.add_option("-p",
                      dest="port",
                      type="int",
                      help="server port (default: %default)",
                      metavar="tcpPort",
                      default=8194)

    (options, args) = parser.parse_args()

    return options


In [7]:
def authorize(session, cId):
    authQueue = blpapi.EventQueue()
    tokenReqId = blpapi.CorrelationId(99)
    session.generateToken(tokenReqId, authQueue)
    done = False
    while not done:
        event = authQueue.nextEvent(1000)
        if event.eventType() != blpapi.Event.TIMEOUT:
            if event.eventType() == blpapi.Event.TOKEN_STATUS:
                for msg in event:
                    if msg.messageType() == TOKEN_SUCCESS:
                        # Token generation success
                        print(msg)
                        if msg.hasElement("token"):
                            token = msg.getElementAsString("token")
                        done = True
                        break
                    elif msg.messageType() == TOKEN_FAILURE:
                        # Token generation failure
                        print(msg)
                        return
                    else:
                        print(msg)
                        break

    identity = session.createIdentity()
    authSvc = session.getService("//blp/apiauth")
    authRequest = authSvc.createAuthorizationRequest()
    authRequest.set("token", token)
    
    session.sendAuthorizationRequest(
        authRequest,
        identity,
        cId,
        authQueue)

    while True:
        # nextEvent() method below is called with a timeout to let
        # the program catch Ctrl-C between arrivals of new events
        event = authQueue.nextEvent(1000)
        if event.eventType() != blpapi.Event.TIMEOUT:
            for msg in event:
                if msg.messageType() == AUTHORIZATION_SUCCESS:
                    # Authorization successful
                    print(msg)
                    return identity
                elif msg.messageType() == AUTHORIZATION_FAILURE:
                    # Authorization failure
                    print(msg)
                    exit()
                else:
                    print(msg)
                    exit()
        else:
            print ("Timeout occurred during authorization")
            exit()


In [8]:
def main():
    host = "69.184.252.34"
    port = 8194
    auth = "APPLICATION"
    appName = "hoopp:Octopus1"
    dirName = ""
    
    #options = parseCmdLine()

    # Fill SessionOptions
    sessionOptions = blpapi.SessionOptions()
    #sessionOptions.setServerHost(options.host)
    #sessionOptions.setServerPort(options.port)
    sessionOptions.setServerHost(host)
    sessionOptions.setServerPort(port)

    # Authentication option
    if auth == "APPLICATION":
        # Set Application Authentication Option
        authOption = "AuthenticationMode=APPLICATION_ONLY;ApplicationAuthenticationType=APPNAME_AND_KEY;ApplicationName="
        authOption += appName
    elif auth == "USER_APP":
        # Set User and Application Authentication Option
        authOption = "AuthenticationMode=USER_AND_APPLICATION;"
        authOption += "AuthenticationType=OS_LOGON;"
        authOption += "ApplicationAuthenticationType=APPNAME_AND_KEY;"
        # ApplicationName is the entry in EMRS.
        authOption += "ApplicationName=" + appName
    elif auth == "DIRSVC":
        # Authenticate user using active directory service property
        authOption = "AuthenticationType=DIRECTORY_SERVICE;"
        authOption += "DirSvcPropertyName=" + dirName
    else:
        # Authenticate user using windows/unix login name
        authOption = "AuthenticationType=OS_LOGON"
    
    sessionOptions.setAuthenticationOptions(authOption)
    
    print ("authOptions = %s" % authOption)

    print ("Connecting to %s:%s" % (host, port))
    # Create a Session
    eventHandler = SessionEventHandler()
    session = blpapi.Session(sessionOptions, eventHandler.processEvent)

    # Start a Session
    if not session.start():
        print ("Failed to start session.")
        return

    try:
        # Open service to get historical data from
        if not session.openService("//blp/refdata"):
            print ("Failed to open //blp/refdata")
            return

        # Open authorization service
        if not session.openService("//blp/apiauth"):
            print ("Failed to open //blp/apiauth")
            return

        g_identityCId = blpapi.CorrelationId(1)
        g_identity = authorize(session, g_identityCId)
        
        if g_identity.getSeatType() == blpapi.Identity.INVALID_SEAT:
            print ("Authentication failed.")
            return
        elif g_identity.getSeatType() == blpapi.Identity.NONBPS:
            print ("Non-BPS seat type can not make historical data request.")
            return
         
        # Obtain previously opened service
        refDataService = session.getService("//blp/refdata")

        # Create and fill the request for the historical data
        request = refDataService.createRequest("HistoricalDataRequest")
        request.getElement("securities").appendValue("EI486050@MSG1 Corp")
        request.getElement("securities").appendValue("EJ120557@MSG1 Corp")
        request.getElement("fields").appendValue("PX_LAST")
        request.getElement("fields").appendValue("OPEN")
        request.set("startDate", "20180101")
        request.set("endDate", "20181231")

        eventQueue = blpapi.EventQueue()
        cId = blpapi.CorrelationId(100)
        print ("Sending Request:", request)
        # Send the request
        session.sendRequest(request, g_identity, cId, eventQueue)

        # Process received events
        while(True):
            # We provide timeout to give the chance for Ctrl+C handling:
            ev = eventQueue.nextEvent(5000)
            if (ev.eventType() == blpapi.Event.PARTIAL_RESPONSE or
                ev.eventType() == blpapi.Event.RESPONSE):
                for msg in ev:
                    if msg.hasElement(RESPONSE_ERROR, True):
                        print ("Request error: %s" % (msg))
                        continue
                    
                    #print msg
                    security_data = msg.getElement('securityData')
                    security_name = security_data.getElement('security')
                    field_data = security_data.getElement('fieldData')
                    
                    print ("Security: %s" % (security_name))
                    for i in range(field_data.numValues()):
                        # get row
                        fields = field_data.getValue(i)
                        if (fields.hasElement("date")):
                            field = fields.getElement("date")
                            print ("\t%s = %s" % (field.name(), field.getValue()))
                        
                        for j in range(fields.numElements()):
                            # process field data
                            field = fields.getElement(j)
                            print ("\t\t%s = %s" % (field.name(), field.getValue()))
                            
            elif (ev.eventType() == blpapi.Event.TIMEOUT):
                print ("Application timeout waiting for response. Canceling request.")
                session.cancel(cId)
                # print all other event messages
                for msg in ev:
                    print (msg)
            else:
                # print all other event messages
                for msg in ev:
                    print (msg)

            if ev.eventType() == blpapi.Event.RESPONSE:
                # Response completly received, so we could exit
                break
            
    finally:
        if g_identity.getSeatType() != blpapi.Identity.INVALID_SEAT: 
            session.cancel(g_identityCId)
        # Stop the session
        session.stop()

In [9]:
if __name__ == "__main__":
    print ("HistoryExample")
    try:
        main()
    except KeyboardInterrupt:
        print ("Ctrl+C pressed. Stopping...")


HistoryExample
authOptions = AuthenticationMode=APPLICATION_ONLY;ApplicationAuthenticationType=APPNAME_AND_KEY;ApplicationName=hoopp:Octopus1
Connecting to 69.184.252.34:8194
2018/01/08 15:26:42: SessionConnectionUp
SessionConnectionUp = {
    server = "69.184.252.34:8194"
    serverId = "16465-108716"
}

2018/01/08 15:26:42: SessionStarted
SessionStarted = {
    initialEndpoints[] = {
        initialEndpoints = {
            address = "69.184.252.34:8194"
        }
    }
}

2018/01/08 15:26:43: ServiceOpened
ServiceOpened = {
    serviceName = "//blp/refdata"
}

2018/01/08 15:26:43: ServiceOpened
ServiceOpened = {
    serviceName = "//blp/apiauth"
}

TokenGenerationSuccess = {
    token = "-v4;lgHKB5HI7d636DLWSiOBYnk7vgOo8Ouk3ZPhCdDgIfPX55huHf0dMZRWPT/UBj4VyOq9uEfX/A2h
rqwM657oBroNpG3hYGh1EtUAyMQ1R+s=;Pfpx/jbALwhoUzODfOufB/SsiiAUGuyDI9ezaIsSdlkkxU0AgEc4OkYP1u61CGCSdqcz2uH2ki+k
sy7l2WMwdWH/EiwjGvSYGCXZLWYaFy8=;"
}

AuthorizationSuccess = {
}

Sending Request: HistoricalDataRequest = {
